# Melodia: A Python Library for Protein Structure and Dynamics Analysis

## Structure Similarity Analysis

In [1]:
import dill
import warnings

import melodia as mel
import seaborn as sns

from os import path

from Bio.PDB.PDBExceptions import PDBConstructionWarning

In [2]:
warnings.filterwarnings("ignore", category=PDBConstructionWarning)

### Parser an alignment in the PIR file format

In [3]:
# Dill can be used for storage
if path.exists('model.dill'):
    with open('model.dill', 'rb') as file:
        align = dill.load(file)
else:
    align = mel.parser_pir_file('model.ali')
    with open('model.dill', 'wb') as file:
        dill.dump(align, file)

In [4]:
palette='OrRd'
colours=6
sns.color_palette(palette, colours)

[(0.9955709342560554, 0.8996539792387543, 0.7629988465974625),
 (0.9921568627450981, 0.8029219530949635, 0.5900192233756247),
 (0.9905113417916186, 0.6576393694732795, 0.4468896578239139),
 (0.9586466743560169, 0.4618992695117263, 0.3103267973856209),
 (0.8704498269896194, 0.2485505574778931, 0.16822760476739718),
 (0.7202306805074971, 0.02435986159169551, 0.015732410611303344)]

In [5]:
mel.cluster_alignment(align=align, threshold=1.3, long=Fals)

In [6]:
mel.save_align_to_ps(align=align, ps_file='model', palette=palette, colours=colours)

In [7]:
mel.save_pymol_script(align=align, pml_file='clusters_model', palette=palette, colours=colours)